In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib.path import Path as MplPath
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- CONFIGURATION ---
project_root = Path.cwd()
fixation_dir = project_root / "Fixation_Results"
excel_path = project_root / "rider_path_shapes.xlsx"  # Excel file with path shapes after every 10 seconds
output_folder = project_root / "Path_Fixation_Analysis" / "riders_variance_updated"
output_folder.mkdir(parents=True, exist_ok=True)
selective_mode = False
selective_participants = ["Charan", "Karim"]
output_csv = output_folder / ("Fixations_On_CyclePath_Summary_selective.csv" if selective_mode else "Fixations_On_CyclePath_Summary_All.csv")
generate_plots = True

canvas_height = 720
canvas_width = 1280

# --- Load Path Shape Excel ---
shape_df = pd.read_excel(excel_path)
shape_df['Participant'] = shape_df['Participant'].str.capitalize()

# --- Load Fixation Files ---
def extract_participant_name(filename):
    return filename.split("_")[0].strip().capitalize()

day_files = list(fixation_dir.glob("*_Daytime_Fixations.csv"))
night_files = list(fixation_dir.glob("*_Nighttime_Fixations.csv"))

day_fixations = {extract_participant_name(f.stem): f for f in day_files}
night_fixations = {extract_participant_name(f.stem): f for f in night_files}

common_participants = sorted(set(day_fixations) & set(night_fixations))
print(f"Participants with both files: {common_participants}")

# --- Generate Participant-Specific Paths from Excel ---
participant_paths = {}
participant_std_polygons = {}

for participant in common_participants:
    part_df = shape_df[shape_df['Participant'] == participant]

    if part_df.empty:
        continue

    # Collect coordinates
    bl_x = part_df['Bottom_Left_X']
    bl_y = part_df['Bottom_Left_Y']
    br_x = part_df['Bottom_Right_X']
    br_y = part_df['Bottom_Right_Y']
    vp_x = part_df['Vanishing_X']
    vp_y = part_df['Vanishing_Y']

    # Calculate means
    bl_mean = [bl_x.mean(), bl_y.mean()]
    br_mean = [br_x.mean(), br_y.mean()]
    vp_mean = [vp_x.mean(), vp_y.mean()]
    polygon_vertices = np.array([bl_mean, br_mean, vp_mean])
    polygon_path = MplPath(polygon_vertices)
    participant_paths[participant] = polygon_path

    # Calculate standard deviations
    bl_std = [bl_x.std(), bl_y.std()]
    br_std = [br_x.std(), br_y.std()]
    vp_std = [vp_x.std(), vp_y.std()]

    # Create inner and outer polygons based on standard deviation
    inner = np.array([
        [bl_mean[0] + bl_std[0], bl_mean[1] + bl_std[1]],
        [br_mean[0] - br_std[0], br_mean[1] + br_std[1]],
        [vp_mean[0] - vp_std[0], vp_mean[1] - vp_std[1]],
    ])

    outer = np.array([
        [bl_mean[0] - bl_std[0], bl_mean[1] - bl_std[1]],
        [br_mean[0] + br_std[0], br_mean[1] - br_std[1]],
        [vp_mean[0] + vp_std[0], vp_mean[1] + vp_std[1]],
    ])

    participant_std_polygons[participant] = {
        'inner': inner,
        'outer': outer,
        'mean': polygon_vertices
    }

# --- Analysis Loop ---
summary = []

for participant in tqdm(selective_participants if selective_mode else common_participants, desc="Processing Participants"):
    if participant not in participant_paths:
        continue

    polygon_path = participant_paths[participant]
    std_data = participant_std_polygons[participant]

    for condition, file in [("Daytime", day_fixations[participant]), ("Nighttime", night_fixations[participant])]:
        df = pd.read_csv(file)
        points = df[['fixation_x', 'fixation_y']].to_numpy()
        points[:, 1] = canvas_height - points[:, 1]  # Invert Y-axis

        inside_mask = polygon_path.contains_points(points)
        fixations_on_path = inside_mask.sum()
        total_fixations = len(df)
        percent_on_path = (fixations_on_path / total_fixations) * 100 if total_fixations > 0 else 0

        summary.append({
            "Participant": participant,
            "Condition": condition,
            "Total_Fixations": total_fixations,
            "Fixations_On_Path": fixations_on_path,
            "Percent_On_Path": percent_on_path
        })

        # --- PLOTTING ---
        if generate_plots:
            plt.figure(figsize=(8, 6))
            plt.scatter(points[:, 0], points[:, 1], c='red', s=10, label='All Fixations')
            plt.scatter(points[inside_mask, 0], points[inside_mask, 1], c='green', s=10, label='Fixations on Path')

            # Mean path
            closed_mean = np.vstack([std_data['mean'], std_data['mean'][0]])
            plt.plot(closed_mean[:, 0], closed_mean[:, 1], c='cyan', lw=2, label='Cycle Path')

            # Inner boundary (−1 STD)
            closed_inner = np.vstack([std_data['inner'], std_data['inner'][0]])
            plt.plot(closed_inner[:, 0], closed_inner[:, 1], linestyle='--', color='gray', label='-1 STD')

            # Outer boundary (+1 STD)
            closed_outer = np.vstack([std_data['outer'], std_data['outer'][0]])
            plt.plot(closed_outer[:, 0], closed_outer[:, 1], linestyle='--', color='black', label='+1 STD')

            plt.xlim(0, canvas_width)
            plt.ylim(0, canvas_height)
            plt.gca().invert_yaxis()
            plt.title(f"{participant} – {condition} Fixations on Cycle Path")
            plt.xlabel("Fixation X (px)")
            plt.ylabel("Fixation Y (px)")
            plt.legend()
            plt.tight_layout()

            plot_path = output_folder / f"{participant}_{condition}_FixationsOnCyclePath.png"
            plt.savefig(plot_path, dpi=300)
            plt.close()

# --- SAVE SUMMARY ---
summary_df = pd.DataFrame(summary)
summary_df.to_csv(output_csv, index=False)
print(f"Summary saved to {output_csv}")


In [ ]:
# --- Comparison Plots for Day vs Night ---
comparison_folder = output_folder / "Comparison_Plots"
comparison_folder.mkdir(exist_ok=True)

for participant in tqdm(selective_participants if selective_mode else common_participants, desc="Creating Comparison Plots"):
    try:
        if participant not in participant_paths:
            continue

        day_df = pd.read_csv(day_fixations[participant])
        night_df = pd.read_csv(night_fixations[participant])

        day_points = day_df[['fixation_x', 'fixation_y']].to_numpy()
        night_points = night_df[['fixation_x', 'fixation_y']].to_numpy()

        day_points[:, 1] = canvas_height - day_points[:, 1]
        night_points[:, 1] = canvas_height - night_points[:, 1]

        std_data = participant_std_polygons[participant]
        path_vertices = std_data['mean']
        closed_mean = np.vstack([path_vertices, path_vertices[0]])

        plt.figure(figsize=(8, 6))
        plt.scatter(day_points[:, 0], day_points[:, 1], c='red', s=10, label='Daytime Fixations')
        plt.scatter(night_points[:, 0], night_points[:, 1], c='blue', s=10, label='Nighttime Fixations')

        plt.plot(closed_mean[:, 0], closed_mean[:, 1], c='cyan', lw=2, label='Cycle Path')

        plt.xlim(0, canvas_width)
        plt.ylim(0, canvas_height)
        plt.gca().invert_yaxis()
        plt.title(f"{participant} – Day vs Night Fixations")
        plt.xlabel("Fixation X (px)")
        plt.ylabel("Fixation Y (px)")
        plt.legend()
        plt.tight_layout()

        save_path = comparison_folder / f"{participant}_Comparison_Day_vs_Night.png"
        plt.savefig(save_path, dpi=300)
        plt.close()

    except Exception as e:
        print(f"Could not generate comparison for {participant}: {e}")
